In [71]:
import pandas as pd
from tqdm import tqdm
import json
import numpy as np
import matplotlib.pyplot as plt

In [72]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from wordcloud import WordCloud, STOPWORDS
from matplotlib import rcParams
import operator

In [73]:
import warnings
warnings.filterwarnings('ignore')

## user.json

See older versions of the file to retrieve the code!

## CSV file from 'business.json'

In [74]:
# Reading in 'extacted_restaurants_from_business.csv'
rest_biz_csvTOpd = pd.read_csv('extracted_restaurants_from_business.csv')

In [75]:
# Business IDs for the restaurants
restaurant_id_list = rest_biz_csvTOpd.business_id.tolist()
restaurant_id_list[:5]

['QXAEGFB4oINsVuTFxEYKFQ',
 'gnKjwL_1w79qoiV3IC_xQQ',
 '1Dfx3zM-rW4n-31KeC8sJg',
 'fweCYi8FmbJXHCqLnwuk8w',
 'PZ-LZzSlhSe9utkQYU8pFg']

In [76]:
print('Number of restaurants in the original dataset:',len(restaurant_id_list))

Number of restaurants in the original dataset: 59371


## CSV file from 'review.json'

In [77]:
# Reading in 'extracted_restaurants_from_review.csv'
rest_review_csvTOpd = pd.read_csv('extracted_restaurants_from_review.csv')

In [78]:
# Rename column containing the old indices
rest_review_csvTOpd.rename(columns={"Unnamed: 0": "Index_From_review.JSON"}).head()

,Index_From_review.JSON,review_id,business_id,stars,text
0,3,yi0R0Ugj_xUx_Nek0-_Qig,ikCg8xy5JIg_NGPx-MSIDA,5.0,Went in for a lunch. Steak sandwich was delici...
1,5,fdiNeiN_hoCxCMy2wTRW9g,eU_713ec6fTGNO4BegRaww,4.0,I'll be the first to admit that I was not exci...
2,6,G7XHMxG0bx9oBJNECG4IFg,3fw2X5bZYeW9xCz_zGhOHg,3.0,Tracy dessert had a big name in Hong Kong and ...
3,7,8e9HxxLjjqc9ez5ezzN7iQ,zvO-PJCpNk4fgAVUnExYAA,1.0,This place has gone down hill. Clearly they h...
4,10,kbtscdyz6lvrtGjD1quQTg,8mIrX_LrOnAqWsB5JrOojQ,4.0,"Like walking back in time, every Saturday morn..."


# Introduction

* We wanted to look to see if potentially we would be able to figure out whether we can identify false reviews that are becoming more common on sites like Amazon.

# Exploratory Data Analysis of reviews.json

## Metrics

In [79]:
def all_metrics(true_labels, predicted_labels):
    print('Accuracy:', np.round(metrics.accuracy_score(true_labels, predicted_labels), 4))
    print('Precision:', np.round(metrics.precision_score(true_labels, predicted_labels, average='weighted'), 4))
    print('Recall:', np.round(metrics.recall_score(true_labels, predicted_labels, average='weighted'), 4))
    print('F1-Score:', np.round(metrics.f1_score(true_labels, predicted_labels,average='weighted'), 4))

### Number of Reviews

In [80]:
rest_review_length = len(rest_review_csvTOpd)
print("There are {} restaurant reviews.".format(rest_review_length))

There are 4201685 restaurant reviews.


In [ ]:
one_star = rest_review_csvTOpd.loc[rest_review_csvTOpd['stars'] == 1.0]
two_star = rest_review_csvTOpd.loc[rest_review_csvTOpd['stars'] == 2.0]
three_star = rest_review_csvTOpd.loc[rest_review_csvTOpd['stars'] == 3.0]
four_star = rest_review_csvTOpd.loc[rest_review_csvTOpd['stars'] == 4.0]
five_star = rest_review_csvTOpd.loc[rest_review_csvTOpd['stars'] == 5.0]

In [ ]:
# Figuring out what percentage of the "rest_review_csvTOpd" should be training data, and what percentage should be test data.
num_train_data = int(rest_review_length * 0.8)
print("For 80% of the data, there should be {} reviews.".format(num_train_data))

# Modeling

## Modeling with 5 Labels

### Histogram of All Reviews in review.json

In [ ]:
plt.figure(figsize=(7,7))
plt.hist(rest_review_csvTOpd.stars, bins=np.arange(10)-0.5, edgecolor='black', linewidth=1.2)
plt.xlim(0.4, 5.6)
plt.title("Histogram of Ratings")
plt.xlabel("Star Rating")
plt.ylabel("Frequency")
plt.show()

### Data

In [ ]:
# Train data for FIVE labels with equal amounts of each rating.
train_data = pd.concat([one_star.text[:1600], two_star.text[:1600], three_star.text[:1600], four_star.text[:1600], five_star.text[:1600]])
train_labels = pd.concat([one_star.stars[:1600], two_star.stars[:1600], three_star.stars[:1600], four_star.stars[:1600], five_star.stars[:1600]])

# Test data for FIVE labels with equal amounts of each rating.
test_data = pd.concat([one_star.text[1600:1700], two_star.text[1600:1700], three_star.text[1600:1700], four_star.text[1600:1700], five_star.text[1600:1700]])
test_labels = pd.concat([one_star.stars[1600:1700], two_star.stars[1600:1700], three_star.stars[1600:1700], four_star.stars[1600:1700], five_star.stars[1600:1700]])

### Histogram of Train Data

In [ ]:
plt.figure(figsize=(7,7))
plt.hist(train_labels, bins=np.arange(10)-0.5, edgecolor='black', linewidth=1.2)
plt.xlim(0.4, 5.6)
plt.title("Histogram of Train Data Ratings")
plt.xlabel("Star Rating")
plt.ylabel("Frequency")
plt.show()

### CountVectorizer

In [ ]:
# Fit CountVectorizer with your text data
cvector = CountVectorizer()
# train_data = testset.text.tolist()
vtrain = cvector.fit_transform(train_data)
vtest = cvector.transform(test_data)

In [ ]:
# Train LogisticRegression Classifier
lr_classifier = LogisticRegression(C=0.09)
lr_classifier.fit(vtrain, train_labels)
train_predicted_labels = lr_classifier.predict(vtest)
all_metrics(test_labels, train_predicted_labels)

#### Confusion Matrix for CountVectorizer

In [ ]:
print(confusion_matrix(test_labels, train_predicted_labels))

* Note: Rows reflect the actual values. Columns reflect the predicted values.

* Fairly standard confusion matrix with the majority of test_labels and predicted_labels in the same game.

### TfidfVctorizer

In [ ]:
# Fit TfidfVectorizer with your text data
tvector = TfidfVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1,2))
vtrain = tvector.fit_transform(train_data)
vtest = tvector.transform(test_data)
tvector.fit(test_data)

In [ ]:
# Train LogisticRegression Classifier
lr_classifier = LogisticRegression(C=0.09)
lr_classifier.fit(vtrain, train_labels)
train_predicted_labels = lr_classifier.predict(vtest)
all_metrics(test_labels, train_predicted_labels)

#### Confusion Matrix for TfidfVectorizer

In [ ]:
print(confusion_matrix(test_labels, train_predicted_labels))

* What's interesting about this particular matrix is the 4th row. About 38 reviews are predicted to be 5-star reviews, but in actuality they are 4-star reviews of the restaurant. The number of predicted 5-star reviews for ones that are actually 4-star reviews shows that it is difficult to distinguish a 4-star review from a five star one. One possible reason for this is because people often reserve a 5-star style meal for truly exceptional meals. Thus even though they talk about positive aspects of the meal in their 4-star rating, they could be giving the review 4-stars just because it wasn't the "best". 

* This result prompted us to think about working with less labels rather than the current schema of 5 labels, one for each star rating. Perhaps with less labels, it will be much clearer to distinguish clearer difference between labels.

## Modeling with Simplified Labels

### Data

In [ ]:
# Train data for 3 labels with equal amounts of each rating.
train_data = pd.concat([one_star.text[:1600], two_star.text[:1600], three_star.text[:1600], four_star.text[:1600], five_star.text[:1600]])
train_labels = pd.concat([one_star.stars[:1600], one_star.stars[:1600], three_star.stars[:1600], five_star.stars[:1600], five_star.stars[:1600]])

test_data = pd.concat([one_star.text[1600:1700], two_star.text[1600:1700], three_star.text[1600:1700], four_star.text[1600:1700], five_star.text[1600:1700]])
test_labels = pd.concat([one_star.stars[1600:1700], one_star.stars[1600:1700], three_star.stars[1600:1700], five_star.stars[1600:1700], five_star.stars[1600:1700]])

* The simplified data essentially simplified the 5-star rating system into three bins: bad reviews, neutral reviews, and good reviews.

### Histogram of Simplified Train Data

In [ ]:
plt.figure(figsize=(7,7))
plt.hist(train_labels, bins=np.arange(10)-0.5, edgecolor='black', linewidth=1.2)
plt.xlim(0.4, 5.6)
plt.title("Histogram of Train Data Ratings")
plt.xlabel("Rating")
plt.ylabel("Frequency")
plt.show()

* Here, the 1-star and 2-star ratings were the bad reviews, the 3-star rating consisted of the neutral reviews, and the 4-star and the 5-star ratings were the good reviews.

### CountVectorizer

In [ ]:
# Fit CountVectorizer with your text data
cvector = CountVectorizer()
vtrain = cvector.fit_transform(train_data)
vtest = cvector.transform(test_data)

In [ ]:
# Train LogisticRegression Classifier
lr_classifier = LogisticRegression(C=0.09)
lr_classifier.fit(vtrain, train_labels)
train_predicted_labels = lr_classifier.predict(vtest)
all_metrics(test_labels, train_predicted_labels)

#### Confusion Matrix for CountVectorizer

In [ ]:
print(confusion_matrix(test_labels, train_predicted_labels))

## Sample Predictions

In [ ]:
# Fit CountVectorizer with your text data
cvector = CountVectorizer()
vtrain = cvector.fit_transform(train_data)
vtest = cvector.transform(test_data)

In [ ]:
# Train LogisticRegression Classifier
lr_classifier = LogisticRegression(C=0.09)
lr_classifier.fit(vtrain, train_labels)
train_predicted_labels = lr_classifier.predict(vtest)
all_metrics(test_labels, train_predicted_labels)

In [ ]:
# Predict new sentence #1.
test_sentence = ["I hate this place"]
new_test = cvector.transform(test_sentence)
lr_classifier.predict(new_test)
lr_classifier.predict_proba(new_test)

In [ ]:
# Predict new sentence #2.
test_sentence = ["Great place"]
new_test = cvector.transform(test_sentence)
lr_classifier.predict(new_test)
lr_classifier.predict_proba(new_test)

For sentence #1, the model predicts the review to be a negative review, whereas the model predicts that sentence #2 is most likely a positive review.

# Predictions Without Neutral Reviews

In [ ]:
# Redoing Analysis with Neutral Reviews
NO3_train_data = pd.concat([one_star.text[:1600], two_star.text[:1600], four_star.text[:1600], five_star.text[:1600]])
NO3_train_labels = pd.concat([one_star.stars[:1600], one_star.stars[:1600], five_star.stars[:1600], five_star.stars[:1600]])

NO3_test_data = pd.concat([one_star.text[1600:1700], two_star.text[1600:1700], four_star.text[1600:1700], five_star.text[1600:1700]])
NO3_test_labels = pd.concat([one_star.stars[1600:1700], one_star.stars[1600:1700], five_star.stars[1600:1700], five_star.stars[1600:1700]])

### CountVectorizer

In [ ]:
# Fit CountVectorizer with your text data
cvector = CountVectorizer()
v_NO3_train = cvector.fit_transform(NO3_train_data)
v_NO3_test = cvector.transform(NO3_test_data)

In [ ]:
# Train LogisticRegression Classifier
lr_classifier = LogisticRegression(C=0.08)
lr_classifier.fit(v_NO3_train, NO3_train_labels)
train_predicted_labels = lr_classifier.predict(v_NO3_test)
all_metrics(NO3_test_labels, train_predicted_labels)

#### Confusion Matrix for CountVectorizer

In [ ]:
print(confusion_matrix(NO3_test_labels, train_predicted_labels))

### TfidfVctorizer

In [ ]:
# Fit TfidfVectorizer with your text data
tvector = TfidfVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1,2))
v_NO3_train = tvector.fit_transform(NO3_train_data)
v_NO3_test = tvector.transform(NO3_test_data)

In [ ]:
# Train LogisticRegression Classifier
lr_classifier = LogisticRegression(C=0.09)
lr_classifier.fit(v_NO3_train, NO3_train_labels)
train_predicted_labels = lr_classifier.predict(v_NO3_test)
all_metrics(NO3_test_labels, train_predicted_labels)

#### Confusion Matrix for TfidfVectorizer

In [ ]:
print(confusion_matrix(NO3_test_labels, train_predicted_labels))

* So if we throw out the moderate data (the 3-star reviews), it is clear that there is a difference between positive and negative reviews. What are some of those differences in vocabulary though?

# Word Cloud

* Since we're looking to compare reviews written from either end of the spectrum of restraunt ratings, the data that we're looking to train will only consist of the extremes. 

In [ ]:
cloud_train_data = pd.concat([one_star.text[:1600], five_star.text[:1600]]).tolist()
cloud_train_labels = pd.concat([one_star.stars[:1600], five_star.stars[:1600]])

## General Word Cloud

In [ ]:
# Stopwords are words that are commonly used that have been purposefully programmed to be ignored.
stopwords = STOPWORDS
stopwords.add('food')
stopwords.add('place')

In [ ]:
# Here we're looking to 
general = " ".join(cloud_train_data)
wordcloud = WordCloud(stopwords = stopwords, width=1600, height=800, max_font_size=200).generate(general)

In [ ]:
rcParams['figure.figsize'] = 40, 40
plt.imshow(wordcloud, interpolation= 'bilinear')
plt.axis("off")
plt.show()

## General Word Cloud with Positive Reviews

In [ ]:
# Positive Reviews Dataset
positive_reviews = five_star.text[1600:2000].tolist()
positive = " ".join(positive_reviews)

In [ ]:
wordcloud = WordCloud(stopwords = stopwords, width=1600, height=800, max_font_size=200).generate(positive)

In [ ]:
rcParams['figure.figsize'] = 40, 40
plt.imshow(wordcloud, interpolation= 'bilinear')
plt.axis("off")
plt.show()

## General Word Cloud with Negative Reviews

In [ ]:
# Negative Reviews Dataset
negative_reviews = one_star.text[1600:2000].tolist()
negative = " ".join(negative_reviews)

In [ ]:
wordcloud = WordCloud(stopwords = stopwords,width=1600, height=800, max_font_size=200).generate(negative)

In [ ]:
rcParams['figure.figsize'] = 40, 40
plt.imshow(wordcloud, interpolation= 'bilinear')
plt.axis("off")
plt.show()

# Count Vectorizer

In [ ]:
cvector = CountVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1,2))
cvector.fit(cloud_train_data)

In [ ]:
len(cvector.get_feature_names())

## Positive Reviews

In [ ]:
pos_matrix = cvector.transform(positive_reviews)
pos_words = pos_matrix.sum(axis=0)
pos_words_freq = [(word, pos_words[0, idx]) for word, idx in cvector.vocabulary_.items()]
pos_tf = pd.DataFrame(list(sorted(pos_words_freq, key = lambda x: x[1], reverse=True)),columns=['Terms','positive'])

In [ ]:
positive_unique_terms = pos_tf[~pd.DataFrame(pos_tf.Terms.tolist()).isin(stopwords).any(1)]
positive_unique_terms[:20]

## Negative Reviews

In [ ]:
neg_matrix = cvector.transform(negative_reviews)
neg_words = neg_matrix.sum(axis=0)
neg_words_freq = [(word, neg_words[0, idx]) for word, idx in cvector.vocabulary_.items()]
neg_tf = pd.DataFrame(list(sorted(neg_words_freq, key = lambda x: x[1], reverse=True)),columns=['Terms','negative'])

In [ ]:
negative_unique_terms = neg_tf[~pd.DataFrame(neg_tf.Terms.tolist()).isin(stopwords).any(1)]
negative_unique_terms[:20]

# Tfidf Vectorizer

In [ ]:
tvector = TfidfVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1,2))
tvector.fit(cloud_train_data)

### Positive Reviews

In [ ]:
pos_matrix = tvector.transform(positive_reviews)
pos_words = pos_matrix.sum(axis=0)
pos_words_freq = [(word, pos_words[0, idx]) for word, idx in tvector.vocabulary_.items()]
pos_tf = pd.DataFrame(list(sorted(pos_words_freq, key = lambda x: x[1], reverse=True)),columns=['Terms','positive'])

In [ ]:
positive_unique_terms = pos_tf[~pd.DataFrame(pos_tf.Terms.tolist()).isin(stopwords).any(1)]
positive_unique_terms[:20]

* TF-IDF analysis reveals a more words that are unique to the particular subset of positive reviews. Some words like "great", "good", and "delicious" are easy to understand, but then there are certain features of restaurants that get mentioned a lot too in these positive reviews, like the "service", "place", and the "time", presumably talking about the positive aspects of the particular restaurant.

## Negative Reviews

In [ ]:
neg_matrix = tvector.transform(negative_reviews)
neg_words = neg_matrix.sum(axis=0)
neg_words_freq = [(word, neg_words[0, idx]) for word, idx in tvector.vocabulary_.items()]
neg_tf = pd.DataFrame(list(sorted(neg_words_freq, key = lambda x: x[1], reverse=True)),columns=['Terms','negative'])

In [ ]:
negative_unique_terms = neg_tf[~pd.DataFrame(neg_tf.Terms.tolist()).isin(stopwords).any(1)]
negative_unique_terms[:20]

* On the other hand, the negative reviews focus on aspects of the restaurant like the "service", the "order", the "time", "the food", and the "place", as well as words that describe those aspects, like "bad".